<a target="_blank" href="https://colab.research.google.com/github/Diegorb1329/RecNat-Py/blob/main/Libretas_IRNR/Teledetecci%C3%B3n/%C3%8Dndices%20de%20Vegetaci%C3%B3n%20por%20Municipio.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

---
**Autor:** [Diego Rivera Buendía ](https://hyper-hydrogen-a19.notion.site/Hola-Soy-Diego-Rivera-Buend-a-bb4d9e93c11246748dfc618e69e317ea)

**Licencia:** Apache 2.0

**Acceso:** Libre y gratuito bajo demanda

**Nota:** Este documento puede ser utilizado y compartido libremente, siempre que se haga referencia a su autor original. No se permite el uso comercial sin permiso explícito del autor.
---


# Introducción

**Análisis de imágenes Sentinel-2 con [Google Earth Engine](https://earthengine.google.com/) y [Geemap](https://geemap.org/) en Google Colab**

Esta libreta te guiará a través del proceso de carga y análisis de imágenes satelitales de Sentinel-2 utilizando Google Earth Engine. Cubriremos los siguientes temas:

1. Carga de datos geográficos.
2. Filtrado de imágenes por fecha y cobertura nubosa.
3. Composición de bandas.
4. Cálculo y visualización de NDVI.
5. Visualización avanzada en mapas interactivos.

Un mosaico permite analizar imágenes satelitales para sacar el mayor provecho de estas, [aquí](https://learn.arcgis.com/es/projects/get-started-with-multidimensional-multispectral-imagery/) se muestra cómo se puede hacer este proceso de forma "manual" con ArcGis Pro

# **Importación de Librerías y Autenticación** 📓

In [ ]:
import geemap
import ee

## Autenticación

Primero, necesitamos autenticar nuestra sesión de Google Earth Engine y inicializarla. También instalaremos la librería `geemap`, que facilita la visualización de mapas interactivos en notebooks.

*Puedes consultar la siguiente guía para crear una cuenta de Earth Engine y el establecimiento de un nuevo proyecto*

In [ ]:
# En caso de que Geemap no esté instalado, habilita esta línea de código eliminando el símbolo "#"

#!pip install geemap

In [ ]:
ee.Authenticate()
ee.Initialize(project = "edu-irnr") # Hay que modificar el nombre del proyecto al nombre que se tiene en el proyecto propio

# Carga de Datos 🔢

---

Es necesario establecer la conexión a las colecciones de datos a las que queremos acceder. Para este caso serán:


1.   Imágenes Sentinel 2 de la Agencia Espacial Europea
2.   Límites a nivel municipal de México (Se requiere cargar de forma local esta capa)

Además de estos datos, es posible consultar toda la [colección de datos](https://developers.google.com/earth-engine/datasets/) de Google Earth Engine.

---

In [ ]:
# Cargar la colección de municipios de México
municipios_mexico = ee.FeatureCollection("projects/ee-diego-epp/assets/mun_mex")
# Filtrar por el municipio 'Paso de Ovejas' (Cambiar según área de interés)
municipio = municipios_mexico.filter(ee.Filter.eq('NOM_MUN', 'Paso de Ovejas'))


## Definición de las fechas de interés 📅

In [ ]:
# Definir la fecha de inicio y fin
fecha_inicio = '2024-06-01'
fecha_fin = '2024-07-25'

## Carga de imágenes satelitales sin nubes ⛔🌨

In [ ]:

# Cargar y filtrar la colección de imágenes Sentinel-2
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(fecha_inicio, fecha_fin) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 15)) \
    .filterBounds(municipio)

# Al cambiar el  parámetro "CLOUDY_PIXEL_PERCENTAGE", definimos el porcentaje máximo de nubes que deseamos en nuestras imágenes.
# Bajar el porcentaje de nubes deseado implica, generalmente, disponer de una menor cantidad de imágenes

# Composición de bandas y cálculo de NDVI 🔴🟢
Configuraremos parámetros de visualización y crearemos una vista de las bandas RGB de las imágenes.


---

## ¿Qué es el Índice NDVI (Normalized Difference Vegetation Index)?

¡Puedes revisar este [documento](https://compass.onlinelibrary.wiley.com/doi/10.1111/j.1749-8198.2012.00507.x) para tener más información!

El **Índice de Vegetación de Diferencia Normalizada (NDVI)** es una medida cuantitativa que se utiliza para evaluar la densidad y la salud de la vegetación en una determinada área a partir de imágenes satelitales o aéreas. Utiliza las bandas del espectro visible e infrarrojo cercano para hacer esta evaluación.

### Fórmula del NDVI

El NDVI se calcula usando la siguiente fórmula:

NDVI = \(NIR - Red) / (NIR + Red)

Donde:
- **NIR** representa la reflectancia en el infrarrojo cercano.
- **Red** representa la reflectancia en la luz visible roja.

### Valores del NDVI

El rango de valores del NDVI varía de -1 a +1:
- **Valores cercanos a +1**: indican una alta densidad de vegetación con buena salud.
- **Valores cercanos a 0 y negativos**: sugieren áreas sin vegetación, como rocas, ríos y áreas urbanizadas o suelos desnudos.

### Usos del NDVI

El NDVI es ampliamente utilizado en varias aplicaciones, incluyendo:
- **Agricultura**: para monitorear la salud y el crecimiento de los cultivos a lo largo del tiempo.
- **Gestión de recursos naturales**: para estudiar cambios en los ecosistemas y la cobertura vegetal.
- **Planificación urbana y gestión del territorio**: para analizar la distribución de áreas verdes y su salud.

El NDVI es una herramienta esencial para ecologistas, agricultores, y gestores del territorio, proporcionando una visión clara y medible de la vegetación a gran escala a través de teledetección.

---


In [ ]:
# Verificar si hay imágenes disponibles en la colección
if sentinel2.size().getInfo() == 0:

    print("No hay imágenes disponibles, cambie el rango de fechas")

else:

    # Obtener la primera imagen de la colección
    imagen_composite = sentinel2.first()

    # Calcular NDVI
    ndvi = imagen_composite.normalizedDifference(['B8', 'B4']).rename('NDVI')

    # Puedes añadir aquí el código para otras operaciones de bandas

    print("Imágen a color real y NDVI calculado con éxito.")

Imágen a color real y NDVI calculado con éxito.


In [ ]:
# Parámetros de visualización para las bandas RGB
parametros_vis = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4
}


In [ ]:
# Parámetros de visualización para NDVI
parametros_ndvi = {
    'min': 0,
    'max': 1,
    'palette': ['red', 'yellow', 'lightgreen', 'green']
}

In [ ]:
# Crear el mapa y configurar la vista inicial
Mapa = geemap.Map()
Mapa.centerObject(municipio, 12)
Mapa.add_basemap("SATELLITE")
Mapa.add_basemap("OpenTopoMap")

# Crear capas de mosaico
left_layer = geemap.ee_tile_layer(imagen_composite.clip(municipio), parametros_vis, 'Imagen Sentinel-2 RGB', opacity= 0.85)
right_layer = geemap.ee_tile_layer(ndvi.clip(municipio), parametros_ndvi, 'NDVI', opacity= 0.85)

# Utilizar split_map para mostrar ambas capas lado a lado
Mapa.split_map(left_layer, right_layer)
# Mostrar el mapa

#**Despliegue del mapa** 🗺

In [ ]:
# Mostrar el mapa
Mapa

Map(center=[19.246367725884216, -96.45007000915615], controls=(ZoomControl(options=['position', 'zoom_in_text'…

# Conclusión 🔚

---
A través de este flujo de trabajo nos es posible acceder a imágenes sentinel actualizadas de cualquier municipio en México desde 2017 a la fecha. Este es un análsis sencillo pero partiendo de esta idea general podemos:

1.   Generar análisis multitemporales
2.   Crear una interfáz visual para poder exponer nuestros algoritmos
3.   Probar otros índices de vegetación, algoritmos de clasificación regresión, etc.

---
